In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!',
 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n*   **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker works with dough all day long.\n*   **Figurative Meaning:** "Dough" is a slang term for money.\n\nThe joke implies that the pizza maker quit because he was tired of making so much pizza dough (literal), but the humor comes from the suggestion that he was tired of making so much money (figurative). It\'s a pun!'}

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!', 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n*   **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker works with dough all day long.\n*   **Figurative Meaning:** "Dough" is a slang term for money.\n\nThe joke implies that the pizza maker quit because he was tired of making so much pizza dough (literal), but the humor comes from the suggestion that he was tired of making so much money (figurative). It\'s a pun!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-d897-6062-8002-d07e790bc886'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-30T17:35:37.337251+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-c

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!', 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n*   **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker works with dough all day long.\n*   **Figurative Meaning:** "Dough" is a slang term for money.\n\nThe joke implies that the pizza maker quit because he was tired of making so much pizza dough (literal), but the humor comes from the suggestion that he was tired of making so much money (figurative). It\'s a pun!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-d897-6062-8002-d07e790bc886'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-30T17:35:37.337251+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti blush?\n\nBecause it saw the meatball!',
 'explanation': 'The joke "Why did the spaghetti blush? Because it saw the meatball!" relies on a simple, slightly suggestive play on words. Here\'s the breakdown:\n\n* **Blushing:** Blushing is a physical reaction, typically associated with embarrassment, shyness, or feeling flustered. It\'s a human characteristic.\n\n* **Anthropomorphism:** The joke gives human qualities (the ability to blush) to inanimate objects (spaghetti). This is a common comedic technique.\n\n* **The Suggestion:** The humor comes from the implied, slightly risqué association between the spaghetti and the meatball. The joke hints at a suggestive or romantic encounter between the two. The meatball, in this context, is presented in a way that might cause the spaghetti to feel embarrassed or flustered, hence the "blush." Think of it as a lighthearted, food-based innuendo.\n\nIn essence, the joke is funny because it:\n\n* **I

In [11]:
workflow.get_state(config2).config

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f085cc8-5924-689d-8002-3e245c06c297'}}

In [12]:
list(workflow.get_state_history(config2))[-1].config

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f085cc8-39ed-6672-bfff-51934025c9a2'}}

## Time Travel

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!', 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n*   **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker works with dough all day long.\n*   **Figurative Meaning:** "Dough" is a slang term for money.\n\nThe joke implies that the pizza maker quit because he was tired of making so much pizza dough (literal), but the humor comes from the suggestion that he was tired of making so much money (figurative). It\'s a pun!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-d897-6062-8002-d07e790bc886'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-30T17:35:37.337251+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-

In [14]:
list(workflow.get_state_history(config1))[1].config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f085c7b-ced8-6bae-8001-112d91f43b14'}}

In [17]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f085c7b-ced8-6bae-8001-112d91f43b14"}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f085c7b-ced8-6bae-8001-112d91f43b14'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-30T17:35:36.315588+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-c79f-60bc-8000-e7e3ba06eb6e'}}, tasks=(PregelTask(id='ba497e02-cdf0-beb2-72e7-d09881742825', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'The joke plays on the double meaning of the word "dough."\n\n*   **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker works with dough all day long.\n*   **Figurative Meaning:** "Dough" is a slang term for money.\n

In [18]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f085c7b-ced8-6bae-8001-112d91f43b14"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!',
 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n* **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker handles dough all day.\n\n* **Figurative Meaning:** "Dough" is slang for money. To "get dough" means to earn money.\n\nThe joke implies that the pizza maker quit because he was overwhelmed by the amount of *money* he was earning (getting so much "dough"). It\'s funny because you initially think of the literal meaning of dough (pizza dough), but the punchline reveals the intended meaning is about money. The unexpected shift in meaning is what makes the joke humorous.'}

In [21]:
workflow.get_state(config1).values

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!',
 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n* **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker handles dough all day.\n\n* **Figurative Meaning:** "Dough" is slang for money. To "get dough" means to earn money.\n\nThe joke implies that the pizza maker quit because he was overwhelmed by the amount of *money* he was earning (getting so much "dough"). It\'s funny because you initially think of the literal meaning of dough (pizza dough), but the punchline reveals the intended meaning is about money. The unexpected shift in meaning is what makes the joke humorous.'}

In [22]:
for x in list(workflow.get_state_history(config1))[::-1]:
    print(x.config)
    print(x.values)
    print('-'*20)

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-c79c-67ba-bfff-dda3ef5df583'}}
{}
--------------------
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-c79f-60bc-8000-e7e3ba06eb6e'}}
{'topic': 'pizza'}
--------------------
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-ced8-6bae-8001-112d91f43b14'}}
{'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!'}
--------------------
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-d897-6062-8002-d07e790bc886'}}
{'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!', 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n*   **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker works 

### Update State

In [23]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f085c7b-c79f-60bc-8000-e7e3ba06eb6e", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f085cd6-a5a6-67cf-8001-fc54e1a982c4'}}

In [24]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085cd6-a5a6-67cf-8001-fc54e1a982c4'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-30T18:16:14.758497+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f085c7b-c79f-60bc-8000-e7e3ba06eb6e'}}, tasks=(PregelTask(id='b4a4675c-7f50-9d4d-c13d-806637de8087', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting so much dough!', 'explanation': 'The joke plays on the double meaning of the word "dough."\n\n* **Literal Meaning:** "Dough" is the uncooked mixture of flour, water, and other ingredients used to make pizza crust. A pizza maker handles dough all day.\n\n* *

In [25]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f085cd6-a5a6-67cf-8001-fc54e1a982c4"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling really pressured!',
 'explanation': 'The joke plays on the double meaning of the word "pressured". Here\'s a breakdown:\n\n* **Literal meaning (related to samosas):** Samosas are made by pressing the dough around the filling to form their characteristic shape. They are literally "pressured" during their creation.\n\n* **Figurative meaning (related to therapy):** To feel "pressured" means to feel stressed, overwhelmed, and like you\'re under a lot of strain. This is a common reason why people seek therapy.\n\nThe humor comes from the unexpected connection between the literal pressure involved in making a samosa and the emotional pressure a person might experience. The joke personifies the samosa, giving it human-like feelings of stress related to its own creation. Therefore, it goes to therapy for being pressured (in the literal samosa-making sense, but also implying emotional stress).'}

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling really pressured!', 'explanation': 'The joke plays on the double meaning of the word "pressured". Here\'s a breakdown:\n\n* **Literal meaning (related to samosas):** Samosas are made by pressing the dough around the filling to form their characteristic shape. They are literally "pressured" during their creation.\n\n* **Figurative meaning (related to therapy):** To feel "pressured" means to feel stressed, overwhelmed, and like you\'re under a lot of strain. This is a common reason why people seek therapy.\n\nThe humor comes from the unexpected connection between the literal pressure involved in making a samosa and the emotional pressure a person might experience. The joke personifies the samosa, giving it human-like feelings of stress related to its own creation. Therefore, it goes to therapy for being pressured (in the literal samosa-making sense, but also implying emotional s

## Fault Tolerance

In [27]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [28]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [29]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
import time
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

In [ ]:
# Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))